# 畳み込みニューラルネットワーク１

In [1]:
# 必要なライブラリを読み込む
import numpy as np
import pandas as pd
import math
from keras.datasets import mnist
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [2]:
"""前回のsprintからクラスを参照"""
class Sigmoid:
    """シグモイド"""
    def forward(self, A):
        self.A = A
        return self.sigmoid(A)
    
    def backward(self, dZ):
        _sig = self.sigmoid(self.A)
        return dZ * (1 - _sig)*_sig
    
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-X))

class Tanh:
    """Tanh"""
    def forward(self, A):
        self.A = A
        return np.tanh(A)
    
    def backward(self, dZ):
        return dZ * (1 - (np.tanh(self.A))**2)

class Softmax:
    """ソフトマックス"""
    def forward(self, X):
        self.Z = np.exp(X) / np.sum(np.exp(X), axis=1).reshape(-1,1)
        return self.Z
    
    def backward(self, Y):
        self.loss = self.loss_func(Y)
        return self.Z - Y
    
    def loss_func(self, Y, Z=None):
        if Z is None:
            Z = self.Z
        return (-1)*np.average(np.sum(Y*np.log(Z), axis=1))

class ReLU:
    """ReLU"""
    def forward(self, A):
        self.A = A
        return np.clip(A, 0, None)
    
    def backward(self, dZ):
        return dZ * np.clip(np.sign(self.A), 0, None)

class FC:
    """全結合"""
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.W = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2)
        
    def forward(self, X):
        self.X = X
        A = X@self.W + self.B
        return A
    
    def backward(self, dA):
        dZ = dA@self.W.T
        self.dB = np.sum(dA, axis=0)
        self.dW = self.X.T@dA
        self.optimizer.update(self)
        return dZ

class XavierInitializer:
    """sigmoido,tanhの初期化クラス"""
    def W(self, n_nodes1, n_nodes2):
        self.sigma = math.sqrt(1 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B
    
class HeInitializer():
    """ReLUの初期化クラス"""
    def W(self, n_nodes1, n_nodes2):
        self.sigma = math.sqrt(2 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2)
        return B

class SGD:
    """Stochastic Gradient Descent : 確率的勾配降下法"""
    def __init__(self, lr):
        self.lr = lr
    
    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB
        return

class AdaGrad:
    """学習係数を自動で調整してくれる確率的勾配降下法"""
    def __init__(self, lr):
        self.lr = lr
        self.HW = 1
        self.HB = 1
    
    def update(self, layer):
        self.HW += layer.dW**2
        self.HB += layer.dB**2
        layer.W -= self.lr * np.sqrt(1/self.HW) * layer.dW
        layer.B -= self.lr * np.sqrt(1/self.HB) * layer.dB  
    
class GetMiniBatch:
    """ミニバッチクラス"""
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
        
    def __len__(self):
        return self._stop
    
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1] 
    
    def __iter__(self):
        self._counter = 0
        return self
    
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

class SimpleInitializer:
    """初期化クラス"""
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, *shape):
        W = self.sigma * np.random.randn(*shape)
        return W
    
    def B(self, *shape):
        B = self.sigma * np.random.randn(*shape)
        return B
    


## 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成

チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成してください。基本構造は前のSprintで作成した全結合層のFCクラスと同じになります。なお、重みの初期化に関するクラスは必要に応じて作り変えてください。Xavierの初期値などを使う点は全結合層と同様です。
ここでは パディング は考えず、ストライド も1に固定します。また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応してください

フォワードプロパゲーションの数式は以下のようになります。

$$
a_i=∑^{F−1}_{s=0} x_{(i+s)}w_s+b
$$

$a_i$ : 出力される配列のi番目の値


$F$ : フィルタのサイズ


$x_{(i+s)}$ : 入力の配列の(i+s)番目の値


$w_s$ : 重みの配列のs番目の値


$b$ : バイアス項


すべてスカラーです。

_____________________________________________

次に更新式です。ここがAdaGradなどに置き換えられる点は全結合層と同様です。

$$
w′_s=w_s−α\frac{\partial L}{\partial w_s}
$$
$$
b′=b−α\frac{\partial L}{\partial b}
$$

$\alpha$ : 学習率


$\frac{\partial L}{\partial w_s}$ : $w_s$ に関する損失 $L$ の勾配


$\frac{\partial L}{\partial b}$ : $b$ に関する損失 $L$ の勾配

_______________________________________________

勾配 $\frac{\partial L}{\partial w_s}$ や $\frac{\partial L}{\partial b}$ を求めるためのバックプロパゲーションの数式が以下です。

$$
    \frac{\partial L}{\partial w_s}=∑^{Nout−1}_{i=0} \frac{\partial L}{\partial a_i}x_{(i+s)}
$$
$$
\frac{\partial L}{\partial b}=∑^{Nout−1}_{i=0} \frac{\partial L}{\partial a_i}
$$

$\frac{\partial L}{\partial a_i}$ : 勾配の配列のi番目の値


$N_{out}$ : 出力のサイズ

__________________________________________________

前の層に流す誤差の数式は以下です。

$$
\frac{\partial L}{\partial x_j}=∑^{F−1}_{s=0} \frac{\partial L}{\partial a_{(j−s)}}w_s
$$

$\frac{\partial L}{\partial x_j}$ : 前の層に流す誤差の配列のj番目の値


ただし、 $j-s<0$ または $j-s>N_{out}-1$ のとき $\frac{\partial L}{\partial a_{(j-s)}} =0$ です。

全結合層との大きな違いは、重みが複数の特徴量に対して共有されていることです。この場合は共有されている分の誤差をすべて足すことで勾配を求めます。計算グラフ上での分岐はバックプロパゲーションの際に誤差の足し算をすれば良いことになります。

In [3]:
class SimpleConv1d:
    """
    スクラッチクラス
    """
    """フォワード"""
    def forward(self,x,f,b,s=1,p=0):
        """
        順伝播
        --------
        x : 入力配列
        f : フィルター
        b : バイアス
        s : ストライド
        p : パディング
        """
        h = x.shape[0]
        w = len(x)
        fs = f.shape[0]
        fh = len(f)
        oh = (h + 2*p - fh)/s + 1
        ow = (w + 2*p - fs)/s + 1

        pred =[]
        for i in range(int(ow)):
            ans = 0
            for s in range(int(fs)):
                ans += x[i+s] * f[s]
            ans = ans + b 
            print(ans)
            pred = np.append(pred,ans)
        return pred
    
    """バックプロバケーション"""
    def backward(self,x,f,error_):
        """
        逆伝播
        -------
        x : 入力配列
        f : フィルター
        error_ : 誤差
        """
        # バイアスの勾配
        delta_b = np.array(np.sum(error_))
        
        f_ = f[::-1]
        a_0 = error_[0]
        a_1 = error_[1]
        
        # 逆伝播の計算
        dx3 = a_1 * f[2]        
        dx2 = np.sum(error_*f_[0:2])
        dx1 = np.sum(error_*f_[1:3])
        dx0 = a_0 * f[0]
        
         # 逆伝播の値
        delta_x = np.array([dx0,dx1,dx2,dx3])
        
        
        # 
        dw2 = np.sum(error_*x[2:4])
        dw1 = np.sum(error_*x[1:3])       
        dw0 = np.sum(error_*x[0:2])
              
        # 重みの勾配
        delta_w = np.array([dw0,dw1,dw2])
        
       
        
        return delta_w,delta_x,delta_b
    

## 【問題2】1次元畳み込み後の出力サイズの計算

畳み込みを行うと特徴量の数が変化します。どのように変化するかは以下の数式から求められます。パディングやストライドも含めています。この計算を行う関数を作成してください。

$$
N_{out}=\frac{N_{in}+2P−F}{S}+1
$$

$N_{out}$ : 出力のサイズ（特徴量の数）


$N_{in}$ : 入力のサイズ（特徴量の数）


$P$ : ある方向へのパディングの数


$F$ : フィルタのサイズ


$S$ : ストライドのサイズ

In [4]:
def output_size_calculation(inp,f,p=0,s=1):
    out_ = int((inp + 2*p - f)/s + 1)
    
    return out_

## 【問題3】小さな配列での1次元畳み込み層の実験

次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認してください。

入力x、重みw、バイアスbを次のようにします。

x = np.array([1,2,3,4])
f = np.array([3,5,7])
b = np.array([1])

フォワードプロパゲーションをすると出力は次のようになります。
a = np.array([35, 50])


次にバックプロパゲーションを考えます。誤差は次のようであったとします。

delta_a = np.array([10, 20])

バックプロパゲーションをすると次のような値になります。
delta_b = np.array([30])
delta_w = np.array([50, 80, 110])
delta_x = np.array([30, 110, 170, 140])

In [5]:
x = np.array([1,2,3,4])
f = np.array([3,5,7])
b = np.array([1])
s = 1
p = 0
y = np.array([45,70])
error_ =([10,20])

In [6]:
# インスタンス化
sc_1d = SimpleConv1d()

In [7]:
# 順伝播
sc_1d.forward(x,f,b,s,p)

[35]
[50]


array([35., 50.])

In [8]:
# 逆伝播
db, dw, dx = sc_1d.backward(x, f, error_)
print(dx)
print(dw)
print(db)

30
[ 30 110 170 140]
[ 50  80 110]


## 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成

チャンネル数を1に限定しない1次元畳み込み層のクラスConv1dを作成してください。


例えば以下のようなx, w, bがあった場合は、

x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。\
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。\
b = np.array([1, 2, 3]) # （出力チャンネル数）

出力は次のようになります。

a = np.array([[16, 22], [17, 23], [18, 24]]) # shape(3, 2)で、（出力チャンネル数、特徴量数）である。

In [9]:
class Conv1d:
    """畳み込みクラス
    """
    def __init__(self, b_size, initializer, optimizer, n_in_channels=1, n_out_channels=1, pa=0):
        """コンストラクタ
        b_size : フィルターサイズ
        initializer : 初期化クラス
        optimizer : 最適化手法クラス
        n_in_channels : 入力チャンネル数
        n_out_channels : 出力チャンネル数
        pa : パディング数
        """
        self.b_size = b_size
        self.optimizer = optimizer
        self.pa = pa
        # 重みの初期化
        self.W = initializer.W(n_out_channels, n_in_channels) #, b_size)
        # バイアスの初期化
        self.B = initializer.B(n_out_channels)
        self.n_in_channels = n_in_channels
        self.n_out_channels = n_out_channels
        self.n_out = None
        
    def forward(self, X):
        """順伝播
        X : 入力配列
        """
        self.X1 =np.zeros((self.n_out_channels ,X.shape[0])).astype(int)
        self.s1 =np.zeros((self.n_out_channels ,X.shape[0])).astype(int)
        
        j = 0
        for k in range(self.n_out_channels):
            for i in range(X.shape[0]):
                for j in range(self.n_in_channels):
                    self.s1 [k,i] +=np.sum(X[i,j:j+3] * self.W[k,i])
                
        self.s2 = self.s1 + self.B.reshape(-1,1)
        
        return self.s2
    
    def backward(self, dA):
        """逆伝播
        dA : 逆伝播してきた配列
        """
        # 重みの勾配
        self.dW = np.sum(np.dot(dA, self.X1[:, :, self.b_size-1-self.pa:self.n_in+self.pa, np.newaxis]), axis=-1)
        # バイアスの勾配
        self.dB = np.sum(dA, axis=1)
        # 逆伝播の値計算のためにdAを変形
        self.dA = np.pad(dA, ((0,0), (0, (self.b_size-1))))
        # 出力配列（dX）の計算のためゼロ配列dA1を用意する
        self.dA1 = np.zeros((self.n_out_channels, self.b_size, self.dA.shape[-1]))
        # 重みの長さでループ
        for i in range(self.b_size):
            self.dA1[:, i] = np.roll(self.dA, i, axis=-1)
        dX = np.sum(self.W@self.dA1, axis=0)
        # 重みとバイアスの更新
        self.optimizer.update(self)
        return dX

In [10]:
# インスタンス化
conc_1d = Conv1d(b_size=3, initializer=SimpleInitializer(0.01), optimizer=SGD(0.01), n_in_channels=2, n_out_channels=3, pa=0)

In [11]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]])
# コンストラクタ内で初期化しているけど確認のため再代入
conc_1d.W = np.ones((3, 2, 3), dtype=float)
conc_1d.B = np.array([1, 2, 3], dtype=float)

In [12]:
conc_1d.forward(x)

array([[16., 22.],
       [17., 23.],
       [18., 24.]])

In [13]:
"""ストライド変更用"""
class Conv1d_Arbitrary_Strides:
    """畳み込みクラス
    """
    def __init__(self, b_size, initializer, optimizer, n_in_channels=1, n_out_channels=1, pa=0, stride=1):
        """コンストラクタ
        b_size : フィルターサイズ
        initializer : 初期化クラス
        optimizer : 最適化手法クラス
        n_in_channels : 入力チャンネル数
        n_out_channels : 出力チャンネル数
        pa : パディング数
        stride : ストライド数
        """
        self.b_size = b_size
        self.optimizer = optimizer
        self.pa = pa
        self.stride = stride
        # 重みの初期化
        self.W = initializer.W(n_out_channels, n_in_channels, b_size)
        # バイアスの初期化
        self.B = initializer.B(n_out_channels)
        self.n_in_channels = n_in_channels
        self.n_out_channels = n_out_channels
        self.n_out = None
        
    def forward(self, X):
        """順伝播
        X : 入力配列
        """
        # バッチ数
        self.n_samples = X.shape[0]
        # 入力配列の特徴量数
        self.n_in = X.shape[-1]
        # 出力サイズ
        self.n_out = output_size_calculation(self.n_in, self.b_size, self.pa, self.stride)
        # 計算のためにX変形
        X = X.reshape(self.n_samples, self.n_in_channels, self.n_in)
        # 0埋め実施
        self.X = np.pad(X, ((0,0), (0,0), ((self.b_size-1), 0)))
        # 出力配列（A）の計算のためゼロ配列X1を用意する
        self.X1 = np.zeros((self.n_samples, self.n_in_channels, self.b_size, self.n_in+(self.b_size-1)))
        # 重みの長さでループ
        for i in range(self.b_size):
            # ずらしながら上書き
            self.X1[:, :, i] = np.roll(self.X, -i, axis=-1)
        # 重みとバイアスを考慮して計算
        A = np.sum(self.X1[:, np.newaxis, :, :, self.b_size-1-self.pa:self.n_in+self.pa:self.stride]*self.W[:, :, :, np.newaxis], axis=(2, 3)) + self.B.reshape(-1,1)
        return A
    
    def backward(self, dA):
        """逆伝播
        dA : 逆伝播してきた配列
        """
        # 重みの勾配
        self.dW = np.sum(dA[:, :, np.newaxis, np.newaxis]*self.X1[:, np.newaxis, :, :, self.b_size-1-self.pa:self.n_in+self.pa:self.stride], axis=(0, -1))
        # バイアスの勾配
        self.dB = np.sum(dA, axis=(0, -1))
        # 逆伝播の値計算のためにdAを変形
        self.dA = np.pad(dA, ((0,0), (0,0), (0, (self.b_size-1))))
        # 出力配列（dX）の計算のためゼロ配列dA1を用意する
        self.dA1 = np.zeros((self.n_samples, self.n_out_channels, self.b_size, self.dA.shape[-1]))
        # 重みの長さでループ
        for i in range(self.b_size):
            self.dA1[:, :, i] = np.roll(self.dA, i, axis=-1)
        dX = np.sum(self.W[:, :, :, np.newaxis]*self.dA1[:, :, np.newaxis], axis=(1,3))
        # 重みとバイアスの更新
        self.optimizer.update(self)
        return dX

## 【問題8】学習と推定

これまで使ってきたニューラルネットワークの全結合層の一部をConv1dに置き換えてMNISTを学習・推定し、Accuracyを計算してください。


出力層だけは全結合層をそのまま使ってください。ただし、チャンネルが複数ある状態では全結合層への入力は行えません。その段階でのチャンネルは1になるようにするか、 平滑化 を行なってください。


画像に対しての1次元畳み込みは実用上は行わないことのため、精度は問いません。

In [14]:
# データ読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 画像データ→行データに
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# 正規化
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# onehotベクトル化
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

# 訓練データと評価データに
X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

In [15]:
"""スクラッチ畳み込みニューラルネットワーク"""
class ScratchCNNClassifier:
    
    def __init__(self, num_epoch=10, lr=0.01, batch_size=20, n_features=784, n_nodes1=400, n_nodes2=200, n_output=10, verbose=True, Activater=Tanh, Optimizer=AdaGrad):
        """コンストラクタ
        Parameters
        -----------
        num_epoch : 学習回数
        lr : 学習率
        batch_size : バッチサイズ
        n_features : 特徴量数
        n_nodes1 : 1層目のノード数
        n_nodes2 : 2層目のノード数
        n_output : 出力層の数
        verbose : 仮定出力するか否か
        Activater : 活性化関数
        Optimizer : 最適化手法
        """
        self.num_epoch = num_epoch
        self.lr = lr
        self.verbose = verbose  
        self.batch_size = batch_size 
        self.n_features = n_features 
        self.n_nodes2 = n_nodes2 
        self.n_output = n_output 
        self.Activater = Activater
        if Activater == Sigmoid or Activater == Tanh:
            self.Initializer = XavierInitializer
        elif Activater == ReLU:
            self.Initializer = HeInitializer
        self.Optimizer = Optimizer
    
    def fit(self, X, y, X_val=None, y_val=None):
        """学習
        Parameters
        ----------
        X : 訓練データの説明変数
        y : 訓練データの目的変数
        X_val : 評価データの説明変数
        y_val : 評価データの目的変数
        """        
        # レイヤー初期化
        self.Conv1d_Arbitrary_Strides = Conv1d_Arbitrary_Strides(b_size=7, initializer=SimpleInitializer(0.01), optimizer=self.Optimizer(self.lr), n_in_channels=1, n_out_channels=1, pa=3, stride=2)
        self.Conv1d_Arbitrary_Strides.n_out = output_size_calculation(X.shape[-1], self.Conv1d_Arbitrary_Strides.b_size, self.Conv1d_Arbitrary_Strides.pa, self.Conv1d_Arbitrary_Strides.stride)
        self.activation1 = self.Activater()
        self.FC2 = FC(1*self.Conv1d_Arbitrary_Strides.n_out, self.n_nodes2, self.Initializer(), self.Optimizer(self.lr))
        self.activation2 = self.Activater()
        self.FC3 = FC(self.n_nodes2, self.n_output, self.Initializer(), self.Optimizer(self.lr))
        self.activation3 = Softmax()
        
        # loss配列定義と初期値格納（loss:ミニバッチごとの損失格納  loss_epoch:ミニバッチ学習終了後の全体損失）
        self.loss = []
        self.loss_epoch = [self.activation3.loss_func(y, self.forward_propagation(X))]
        
        # 学習回数分ループ
        for _ in range(self.num_epoch):
            # ミニバッチイテレータ生成
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch_size)
            # イテレータ呼び出し
            for mini_X, mini_y in get_mini_batch:
                # 順伝播
                self.forward_propagation(mini_X)
                # 逆伝播
                self.back_propagation(mini_X, mini_y)
                # 損失記録
                self.loss.append(self.activation3.loss)
            # 損失記録
            self.loss_epoch.append(self.activation3.loss_func(y, self.forward_propagation(X)))
    
    def predict(self, X):
        """予測値出力
        Parameters
        ----------
        X : 説明変数
        """
        return np.argmax(self.forward_propagation(X), axis=1)
    
    def forward_propagation(self, X):
        """順伝播
        Parameters
        ----------
        X : 訓練データの説明変数
        """
        A1 = self.Conv1d_Arbitrary_Strides.forward(X)
        A1 = A1.reshape(A1.shape[0], A1.shape[-1])
        Z1 = self.activation1.forward(A1)
        A2 = self.FC2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.FC3.forward(Z2)
        Z3 = self.activation3.forward(A3)
        return Z3
        
    def back_propagation(self, X, y_true):
        """逆伝播
        Parameters
        ----------
        X : 訓練データの説明変数
        y_true : 正解データ
        """
        dA3 = self.activation3.backward(y_true) 
        dZ2 = self.FC3.backward(dA3)
        dA2 = self.activation2.backward(dZ2)
        dZ1 = self.FC2.backward(dA2)
        dA1 = self.activation1.backward(dZ1)
        dA1 = dA1[:, np.newaxis]
        dZ0 = self.Conv1d_Arbitrary_Strides.backward(dA1) 

In [16]:
cnn = ScratchCNNClassifier(num_epoch=20, lr=0.01, batch_size=20, n_features=784, n_nodes1=400, n_nodes2=400, n_output=10, verbose=True, Activater=Tanh, Optimizer=SGD)
cnn.fit(X_train_[:1000], y_train_[:1000])

In [17]:
y_pred = cnn.predict(X_test)
accuracy_score(y_test, y_pred)

0.8997